In [2]:
from pathlib import Path
import json

#  jobs
import lamf_analysis.code_ocean_scripts.jobs as jobs

import aind_session

# notebook
%load_ext autoreload
%autoreload 2


In [2]:
def thyme_sessions():
    #bad_sessions = ["multiplane-ophys_721291_2024-05-24_10-08-04","multiplane-ophys_721291_2024-05-14_08-24-49"]
    bad_sessions = []
    mouse_sessions = aind_session.get_sessions(subject_id=736963)
    mouse_sessions = [session for session in mouse_sessions if session.id not in bad_sessions]
    mouse_sessions = sorted(mouse_sessions)
    
    return mouse_sessions


def get_asset_type_from_sessions_list(sessions, asset_type: str):
    """Get all assets of a certain type from a list of sessions

    Parameters
    ----------
    sessions : list
        List of aind_session.Session objects
    asset_type : str
        Asset type to filter for (e.g. processed,dlc-eye)
        
    Returns
    -------
    list
        List of aind_session
    """
    assets = []
    for session in sessions:
        for asset in session.data_assets:
            
            # bad assets don't have metadata
            if asset.custom_metadata is None:
                continue
            # raw assets reliably have custom metadata.
            if asset_type == "raw":
                if asset.custom_metadata["data level"] == "raw":
                    assets.append(asset)
            else:
                if asset_type in asset.name:
                    assets.append(asset)
    return assets

In [ ]:
import codeocean
! pip install --upgrade aind-session

In [6]:
sessions = thyme_sessions()
assets = get_asset_type_from_sessions_list(sessions, "raw")
assets_dict = [[{"id": asset.id, "mount": asset.mount}] for asset in assets]
assets_dict

[[{'id': 'b5458427-f446-4308-aba4-609cfeb90acb',
   'mount': 'multiplane-ophys_736963_2024-07-09_09-13-58'}],
 [{'id': '4e8e516a-0cbe-477f-8c19-96913e5aa425',
   'mount': 'multiplane-ophys_736963_2024-07-10_09-02-58'}],
 [{'id': '17a7e310-bc74-4d77-8745-deeee1de9b5c',
   'mount': 'multiplane-ophys_736963_2024-07-11_08-45-05'}],
 [{'id': 'b7ae31ca-71a6-42ea-a467-8e839212fabd',
   'mount': 'multiplane-ophys_736963_2024-07-12_09-20-11'}],
 [{'id': '7d2bc78e-b356-453c-b2e4-007f5b5a92af',
   'mount': 'multiplane-ophys_736963_2024-07-15_09-23-32'}],
 [{'id': 'ec3b0265-25cc-48ad-972e-ef425e877b88',
   'mount': 'multiplane-ophys_736963_2024-07-16_09-17-44'}],
 [{'id': 'bfc7c4c8-efb7-4b67-9967-7d8596465632',
   'mount': 'multiplane-ophys_736963_2024-07-18_08-41-14'}],
 [{'id': 'a0f52a4e-ea76-4d67-b790-5dafb06fff85',
   'mount': 'multiplane-ophys_736963_2024-07-19_09-14-01'}],
 [{'id': '0b6188b4-c0e4-4391-b5b2-0768bd7fbec2',
   'mount': 'multiplane-ophys_736963_2024-07-22_08-54-38'}],
 [{'id': '

In [7]:
jobs.default_dlc_eye(json_output_path="dlc_eye_thyme.json", 
                batch_assets_list=assets_dict)

# zstack reg

In [5]:
assets = [
         [{'id': 'b5458427-f446-4308-aba4-609cfeb90acb',
         'mount': 'multiplane-ophys_736963_2024-07-09_09-13-58'}]
         ]

In [6]:
jobs.default_cortical_zstack_registration(json_output_path="thyme_cortical_zstack_reg_1.json",
                                          batch_assets_list=assets)

## check config

In [20]:
from aind_codeocean_pipeline_monitor.job import PipelineMonitorJob
from aind_codeocean_pipeline_monitor.models import (
    CaptureSettings,
    PipelineMonitorSettings,
)
from codeocean.capsule import Capsules
from codeocean.data_asset import DataAssets
from codeocean.computation import Computations, DataAssetsRunParam, RunParams, NamedRunParam
from codeocean import CodeOcean
import argparse
import logging
import time
import json
import os
import sys
from urllib3.util import Retry
from requests.adapters import HTTPAdapter
import multiprocessing
from multiprocessing import Pool
from typing import List

def load_json_config_dlc(config_path):
    """Load JSON configuration from a file"""
    try:
        with open(config_path, 'r') as f:
            settings = json.load(f)
            print(settings)
            # Create a list of settings from the config
            settings_list = []

            for asset in settings['assets_list']:
                batch_settings = PipelineMonitorSettings.model_validate(
                    {
                        "run_params": {
                        "capsule_id": settings['capsule_id'],
                        "data_assets": [
                            DataAssetsRunParam(
                                id=asset[0]['id'],
                                mount=asset[0]['mount']
                            )
                        ],
                        # "named_parameters": [
                        #     NamedRunParam(param_name=key, value=value)
                        #     for key, value in setting['named_parameters'].items()
                        # ]
                        "named_parameters": []
                    },
                    "capture_settings": CaptureSettings(
                        tags=settings['tags'],
                        process_name_suffix=settings['process_name_suffix']
                    )
                }
            )
                settings_list.append(batch_settings)
        
    except FileNotFoundError:
        logging.error(f"Config file not found: {config_path}")
        sys.exit(1)
    except json.JSONDecodeError as e:
        logging.error(f"Invalid JSON in config file: {e}")
        sys.exit(1)
    except ValueError as e:
        logging.error(f"Invalid configuration: {e}")
        sys.exit(1)

    return settings_list 

settings_list = load_json_config_dlc("/home/matt.davis/code/lamf-analysis/src/lamf_analysis/code_ocean_scripts/dlc_eye_thyme.json")
settings_list


{'capsule_id': '4cf0be83-2245-4bb1-a55c-a78201b14bfe', 'tags': ['derived', 'eye_tracking', 'ophys-mfish'], 'process_name_suffix': 'dlc-eye', 'assets_list': [[{'id': 'b5458427-f446-4308-aba4-609cfeb90acb', 'mount': 'multiplane-ophys_736963_2024-07-09_09-13-58'}], [{'id': '4e8e516a-0cbe-477f-8c19-96913e5aa425', 'mount': 'multiplane-ophys_736963_2024-07-10_09-02-58'}], [{'id': '17a7e310-bc74-4d77-8745-deeee1de9b5c', 'mount': 'multiplane-ophys_736963_2024-07-11_08-45-05'}], [{'id': 'b7ae31ca-71a6-42ea-a467-8e839212fabd', 'mount': 'multiplane-ophys_736963_2024-07-12_09-20-11'}], [{'id': '7d2bc78e-b356-453c-b2e4-007f5b5a92af', 'mount': 'multiplane-ophys_736963_2024-07-15_09-23-32'}], [{'id': 'ec3b0265-25cc-48ad-972e-ef425e877b88', 'mount': 'multiplane-ophys_736963_2024-07-16_09-17-44'}], [{'id': 'bfc7c4c8-efb7-4b67-9967-7d8596465632', 'mount': 'multiplane-ophys_736963_2024-07-18_08-41-14'}], [{'id': 'a0f52a4e-ea76-4d67-b790-5dafb06fff85', 'mount': 'multiplane-ophys_736963_2024-07-19_09-14-01

[PipelineMonitorSettings(alert_url=None, computation_polling_interval=180.0, computation_timeout=None, data_asset_ready_polling_interval=10.0, data_asset_ready_timeout=None, run_params=RunParams(capsule_id='4cf0be83-2245-4bb1-a55c-a78201b14bfe', pipeline_id=None, version=None, resume_run_id=None, data_assets=[DataAssetsRunParam(id='b5458427-f446-4308-aba4-609cfeb90acb', mount='multiplane-ophys_736963_2024-07-09_09-13-58')], parameters=None, named_parameters=[], processes=None), capture_settings=CaptureSettings(name=None, tags=['derived', 'eye_tracking', 'ophys-mfish'], mount=None, description=None, source=None, target=None, custom_metadata=None, data_asset_ids=None, results_info=None, data_description_file_name='data_description.json', process_name_suffix='dlc-eye', process_name_suffix_tz='UTC', permissions=Permissions(users=None, groups=None, everyone=<EveryoneRole.Viewer: 'viewer'>, share_assets=None))),
 PipelineMonitorSettings(alert_url=None, computation_polling_interval=180.0, com

In [21]:
def setup_codeocean_client():
    """Initialize and configure CodeOcean client with retry logic"""
    domain = os.getenv("CODEOCEAN_DOMAIN")
    token = os.getenv("CODEOCEAN_TOKEN")

    if not domain or not token:
        raise ValueError("CODEOCEAN_DOMAIN and CODEOCEAN_TOKEN environment variables must be set")

    client = CodeOcean(domain=domain, token=token)

    # Configure retry logic
    retry = Retry(
        total=5,
        backoff_factor=1,
        status_forcelist=[429, 500, 502, 503, 504],
    )
    adapter = HTTPAdapter(max_retries=retry)
    client.session.mount(domain, adapter)

    # Initialize client components
    client.capsules = Capsules(client.session)
    client.computations = Computations(client.session)
    client.data_assets = DataAssets(client.session)

    return client

client = setup_codeocean_client()

In [22]:

job = PipelineMonitorJob(job_settings=settings_list[0], client=client)
job.run_job()